In [17]:
import os
import sys
import comtypes.client

#set the following flag to True to attach to an existing instance of the program
#otherwise a new instance of the program will be started

AttachToInstance = False

#set the following flag to True to manually specify the path to SAP2000.exe
#this allows for a connection to a version of SAP2000 other than the latest installation
#otherwise the latest installed version of SAP2000 will be launched

SpecifyPath = False

#if the above flag is set to True, specify the path to SAP2000 below
#ProgramPath = 'C:\Program Files\Computers and Structures\SAP2000 24\SAP2000.exe'

#full path to the model
#set it to the desired path of your model

APIPath = r'C:\Users\jorei\Documents\Github\Engineering Fundamentals\Structural\SAP2000'

if not os.path.exists(APIPath):

        try:
            os.makedirs(APIPath)

        except OSError:
            pass

ModelPath = APIPath + os.sep + 'API_1-001.sdb' 

#create API helper object
helper = comtypes.client.CreateObject('SAP2000v1.Helper')
helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)

if AttachToInstance:

    #attach to a running instance of SAP2000
    try:
        #get the active SapObject
            mySapObject = helper.GetObject("CSI.SAP2000.API.SapObject") 

    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)

else:

    if SpecifyPath:
        try:
            #'create an instance of the SAPObject from the specified path
            mySapObject = helper.CreateObject(ProgramPath)

        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)

    else:
        try:
            #create an instance of the SAPObject from the latest installed SAP2000
            mySapObject = helper.CreateObjectProgID("CSI.SAP2000.API.SapObject")

        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)

 

    #start SAP2000 application
    mySapObject.ApplicationStart()





In [18]:
# Create SapModel object
SapModel = mySapObject.SapModel

# Initialize model
SapModel.InitializeNewModel()

# Create new blank model
ret = SapModel.File.NewBlank()

# Define material property
MATERIAL_CONCRETE = 2
ret = SapModel.PropMaterial.SetMaterial('CONC', MATERIAL_CONCRETE)

# Assign isotropic mechanical properties to material
ret = SapModel.PropMaterial.SetMPIsotropic('CONC', 3600, 0.2, 0.0000055)

# Define rectangular frame section property
ret = SapModel.PropFrame.SetRectangle('R1', 'CONC', 12, 12)

# Define frame section property modifiers
ModValue = [1000, 0, 0, 1, 1, 1, 1, 1]
ret = SapModel.PropFrame.SetModifiers('R1', ModValue)

# Switch to k-ft units
kip_ft_F = 4
ret = SapModel.SetPresentUnits(kip_ft_F)

# Add frame object by coordinates
FrameName1, FrameName2, FrameName3 = ' ', ' ', ' '
FrameName1, ret = SapModel.FrameObj.AddByCoord(0, 0, 0, 0, 0, 10, FrameName1, 'R1', '1', 'Global')
FrameName2, ret = SapModel.FrameObj.AddByCoord(0, 0, 10, 8, 0, 16, FrameName2, 'R1', '2', 'Global')
FrameName3, ret = SapModel.FrameObj.AddByCoord(-4, 0, 10, 0, 0, 10, FrameName3, 'R1', '3', 'Global')

# Assign point object restraint at base
PointName1, PointName2 = ' ', ' '
Restraint = [True, True, True, True, False, False]
PointName1, PointName2, ret = SapModel.FrameObj.GetPoints(FrameName1, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName1, Restraint)

# Assign point object restraint at top
Restraint = [True, True, False, False, False, False]
PointName1, PointName2, ret = SapModel.FrameObj.GetPoints(FrameName2, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName2, Restraint)

# Refresh view, update (initialize) zoom
ret = SapModel.View.RefreshView(0, False)

In [19]:
# Add load patterns
LTYPE_OTHER = 8
for i in range(1, 8):
    ret = SapModel.LoadPatterns.Add(str(i), LTYPE_OTHER, 0 if i > 1 else 1, True)

# Assign loading for load pattern 2
PointName1, PointName2, ret = SapModel.FrameObj.GetPoints(FrameName3, PointName1, PointName2)
PointLoadValue = [0, 0, -10, 0, 0, 0]
ret = SapModel.PointObj.SetLoadForce(PointName1, '2', PointLoadValue)
ret = SapModel.FrameObj.SetLoadDistributed(FrameName3, '2', 1, 10, 0, 1, 1.8, 1.8)

# Assign loading for load pattern 3
PointName1, PointName2, ret = SapModel.FrameObj.GetPoints(FrameName3, PointName1, PointName2)
PointLoadValue = [0, 0, -17.2, 0, -54.4, 0]
ret = SapModel.PointObj.SetLoadForce(PointName2, '3', PointLoadValue)

# Assign loading for load pattern 4
ret = SapModel.FrameObj.SetLoadDistributed(FrameName2, '4', 1, 11, 0, 1, 2, 2)

# Assign loading for load pattern 5
ret = SapModel.FrameObj.SetLoadDistributed(FrameName1, '5', 1, 2, 0, 1, 2, 2, 'Local')
ret = SapModel.FrameObj.SetLoadDistributed(FrameName2, '5', 1, 2, 0, 1, -2, -2, 'Local')

# Assign loading for load pattern 6
ret = SapModel.FrameObj.SetLoadDistributed(FrameName1, '6', 1, 2, 0, 1, 0.9984, 0.3744, 'Local')
ret = SapModel.FrameObj.SetLoadDistributed(FrameName2, '6', 1, 2, 0, 1, -0.3744, 0, 'Local')

# Assign loading for load pattern 7
ret = SapModel.FrameObj.SetLoadPoint(FrameName2, '7', 1, 2, 0.5, -15, 'Local')

# Switch to k-in units
kip_in_F = 3
ret = SapModel.SetPresentUnits(kip_in_F)

# Save model
ret = SapModel.File.Save(ModelPath)



In [20]:
# Run model (this will create the analysis model)
ret = SapModel.Analyze.RunAnalysis()

# Initialize for Sap2000 results
SapResult = [0, 0, 0, 0, 0, 0, 0]
PointName1, PointName2, ret = SapModel.FrameObj.GetPoints(FrameName2, PointName1, PointName2)

In [21]:
# Initialize for Sap2000 results
SapResult = [0, 0, 0, 0, 0, 0, 0]
PointName1, PointName2, ret = SapModel.FrameObj.GetPoints(FrameName2, PointName1, PointName2)

# Get Sap2000 results for load cases 1 through 7
for i in range(7):
    NumberResults = 0
    Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3 = [], [], [], [], [], [], [], [], [], [], []
    ObjectElm = 0
    ret = SapModel.Results.Setup.DeselectAllCasesAndCombosForOutput()
    ret = SapModel.Results.Setup.SetCaseSelectedForOutput(str(i+1))

    if i <= 3:
        NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3, ret = SapModel.Results.JointDispl(PointName2, ObjectElm, NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3)
        SapResult[i] = U3[0]
    else:
        NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3, ret = SapModel.Results.JointDispl(PointName1, ObjectElm, NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3)
        SapResult[i] = U1[0]

# Close Sap2000
ret = mySapObject.ApplicationExit(False)
SapModel = None
mySapObject = None

# Fill independent results
IndResult = [-0.02639, 0.06296, 0.06296, -0.2963, 0.3125, 0.11556, 0.00651]

# Fill percent difference
PercentDiff = [0, 0, 0, 0, 0, 0, 0]
for i in range(7):
    PercentDiff[i] = (SapResult[i] / IndResult[i]) - 1

# Display results
for i in range(7):
    print("\n", SapResult[i], "\n", IndResult[i], "\n", PercentDiff[i])


 -0.026389708334779615 
 -0.02639 
 -1.1052111420384314e-05

 0.06295961111456187 
 0.06296 
 -6.176706450644609e-06

 0.06295961111456187 
 0.06296 
 -6.176706450644609e-06

 -0.296301851868091 
 -0.2963 
 6.249976682415337e-06

 0.3125018518689788 
 0.3125 
 5.9259807321243585e-06

 0.11555582519151834 
 0.11556 
 -3.612676083120103e-05

 0.006511371528134643 
 0.00651 
 0.00021068020501435747
